In [1]:
import geopandas as gpd
import fiona
import h3pandas
from scripts.load_gbif import get_dataframe

[['geometry', 'Contagem de individuos', 'Estado']]

In [8]:
observacoes = get_dataframe()[['geometry','Contagem de individuos','Estado']]

c:\Users\kenji\dev\web-scraping-images-vinacea\web-scraping-images-vinacea\web-scraping-venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [11]:
observacoes = gpd.GeoDataFrame(observacoes) 

In [12]:
observacoes.to_file("assets/OUTPUT/gbif.shp")

C:\Users\kenji\AppData\Local\Temp\ipykernel_15640\1841821020.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  observacoes.to_file("assets/OUTPUT/gbif.shp")


In [ ]:

# Caminho para o arquivo GeoPackage
caminho_arquivo_gpkg = 'assets/FEATURES/BRASIL_GEO/bc250_2023_11_23.gpkg'

# Listar camadas disponíveis no arquivo GeoPackage
camadas_disponiveis = fiona.listlayers(caminho_arquivo_gpkg)
layer = "rel_ponto_cotado_altimetrico_p"

In [ ]:
df = gpd.read_file("assets/UTILS/BR_MASKED/BR_MASK.geojson")


In [ ]:
df['geometry'][0].bounds

In [ ]:
import geopandas as gpd
from shapely.geometry import Polygon
import numpy as np

# Definir o polígono original (substitua com seus próprios dados)
poligono = df['geometry'][0]

# Definir o limite da malha quadricular (tamanho dos quadrados)
size = 1 # tamanho dos quadrados

# Calcular o limite da grade
x_min, y_min, x_max, y_max = poligono.bounds

# Criar os limites da grade
x_range = range(int(x_min), int(x_max) + size, size)
y_range = range(int(y_min), int(y_max) + size, size)

# Criar uma lista de quadrados
quadrados = []
for x in x_range:
    for y in y_range:
        quadrado = Polygon([(x, y), (x+size, y), (x+size, y+size), (x, y+size)])
        quadrados.append(quadrado)

# Criar um GeoDataFrame para a grade
grade = gpd.GeoDataFrame({'geometry': quadrados})

# Intersectar a grade com o polígono original
grade = grade[grade.intersects(poligono)]


In [ ]:
grade

In [ ]:
# Inicializar uma lista para armazenar as contagens de ocorrências
contagens = []

# Iterar sobre cada polígono
for index, poligono in grade.iterrows():
    # Filtrar pontos que estão contidos no polígono atual
    observacoes = observacoes[poligono.geometry.within(poligono.geometry)]
    # Calcular a contagem de ocorrências
    contagem_ocorrencias = observacoes['Contagem de individuos'].sum()  # Assumindo que 'contagem' é a coluna de contagem no DataFrame de pontos
    # Adicionar a contagem à lista de contagens
    contagens.append(contagem_ocorrencias)
    
# Adicionar a lista de contagens como uma nova coluna ao GeoDataFrame de polígonos
grade['contagem_ocorrencias'] = contagens

In [ ]:
MASK = gpd.read_file("assets/UTILS/BR_MASKED/BR_MASK.geojson")

In [ ]:
TESTE = gpd.read_file(caminho_arquivo_gpkg, layer=layer)
if TESTE.crs is None:
    print("NONE")

In [ ]:
MASK = gpd.read_file("assets/UTILS/BR_MASKED/BR_MASK.geojson")['geometry']

In [ ]:
TESTE = gpd.read_file(caminho_arquivo_gpkg, layer=layer, mask=MASK)
if TESTE.crs is None:
    print("NONE")

In [ ]:
TESTEh3 = TESTE.h3.geo_to_h3(9)

In [ ]:
TESTE['tipoelemnat'].unique()

In [ ]:
TESTE = TESTE.to_crs(4326)

In [ ]:
if TESTE.crs is None:
    TESTE.crs = 4326

# Transformar para a projeção desejada
TESTE = TESTE.to_crs(4326)

In [ ]:
resultado_filtrado = TESTE[TESTE.geometry.intersects(MASK.unary_union)]

In [ ]:
resultado_filtrado.head()

In [ ]:
TESTE.plot()

In [ ]:
INFOS_UFS = gpd.read_file("assets\FEATURES\MALHAS\BR_UF_2022.shp")

In [ ]:
UFS = ['RJ','SP','MG','ES','PR','SC','RS']
INFOS_UFS = INFOS_UFS[['SIGLA_UF','geometry']]
INFOS_UFS = INFOS_UFS[INFOS_UFS['SIGLA_UF'].isin(UFS)]
INFOS_UFS = INFOS_UFS.to_crs(4326)
INFOS_UFS.to_file("assets/UTILS/BR_MASKED/BR_MASK.geojson", driver='GeoJSON')